In [1]:
!pip install Pillow cryptography


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\ray81\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [2]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives.asymmetric.utils import Prehashed

# Generate a private key for signing
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)

# Save the private key
with open("private_key.pem", "wb") as key_file:
    key_file.write(
        private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        )
    )

# Generate a public key for verification
public_key = private_key.public_key()

# Save the public key
with open("public_key.pem", "wb") as key_file:
    key_file.write(
        public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
    )

# Function to create a digital signature
def create_signature(image_path, private_key):
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    # Create a hash of the image
    image_hash = hashes.Hash(hashes.SHA256())
    image_hash.update(image_data)
    digest = image_hash.finalize()

    # Sign the hash with the private key
    signature = private_key.sign(
        digest,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        Prehashed(hashes.SHA256())
    )

    return signature

# Create a digital signature
signature = create_signature("photo.jpg", private_key)

# Save the signature to a file
with open("signature.bin", "wb") as signature_file:
    signature_file.write(signature)


FileNotFoundError: [Errno 2] No such file or directory: 'photo.jpg'

In [3]:
#vertify digital signature

from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes, serialization

# Function to verify a digital signature
def verify_signature(image_path, signature, public_key):
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    # Create a hash of the image
    image_hash = hashes.Hash(hashes.SHA256())
    image_hash.update(image_data)
    digest = image_hash.finalize()

    # Verify the signature with the public key
    try:
        public_key.verify(
            signature,
            digest,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            Prehashed(hashes.SHA256())
        )
        print("Signature is valid.")
    except Exception as e:
        print("Signature verification failed:", str(e))

# Load the public key
with open("public_key.pem", "rb") as key_file:
    public_key = serialization.load_pem_public_key(key_file.read())

# Load the signature
with open("signature.bin", "rb") as signature_file:
    signature = signature_file.read()

# Verify the signature
verify_signature("photo.jpg", signature, public_key)


FileNotFoundError: [Errno 2] No such file or directory: 'signature.bin'

from PIL import Image, ImageDraw, ImageFont

def add_signature(image_path, signature_text, output_path):
    # Open an image file
    with Image.open(image_path) as img:
        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype("arial.ttf", 50)  # Adjust font and size as needed

        # Position for the signature (bottom right corner)
        text_width, text_height = draw.textsize(signature_text, font)
        width, height = img.size
        x = width - text_width - 10
        y = height - text_height - 10

        # Add the signature to the image
        draw.text((x, y), signature_text, font=font, fill=(255, 255, 255))

        # Save the image with the signature
        img.save(output_path)

# Example usage
add_signature("photo.jpg", "John Doe", "signed_photo.jpg")
